In [1]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt, matplotlib.image as mpimg
%matplotlib inline

labeled_images = pd.read_csv('train.csv')
images = labeled_images.iloc[:,1:]
labels = labeled_images.iloc[:,:1]
def relu(x):
    return np.maximum(0,x)
def relu_dev(x):
    return np.maximum(0,np.sign(x))
def softmax(x):
    A = np.exp(x)
    s = np.sum(x, axis = 1).reshape((A.shape[0], 1))
    A = (A / s) + 1e-15
    return A
def softmax_dev(A_froward,Y):
    return A_froward-Y
    

def forward(A_past, W, b, catches, activ_func):
    Z = np.dot(X, W) + b
    catches.append(Z)
    if activ_func=="relu" :
        A=relu(Z)
        
    if activ_func=="softmax" :  
        A = np.exp(Z)
        s = np.sum(A, axis = 1).reshape((A_past.shape[0], 1))
        A = (A / s) + 1e-15
    
    return A

def backward(da_forward, W, b, catches,Layer,active_func):
    Z_L=catches[Layer]
    if activ_func=="relu" :
        active_dev=relu_dev(Z_L)
    if activ_func=="softmax" : 
         softmax_dev(da_forward,Y)
    dZ = A - Y
    dW = np.dot(X.T, dZ) / X.shape[0]
    db = np.sum(dZ, axis = 0) / X.shape[0]
    
    return dW, db

def update(X, Y, W, b, output_size, learning_rate):
    A = forward(X, W, b, output_size)
    dW, db = backward(X, Y, A)
    
    J = -np.trace(np.dot(Y, np.log(A).T)) / X.shape[0]
    
    return W - learning_rate * dW, b - learning_rate * db, J

def predict(X, W, b, output_size):
    A = forward(X, W, b, output_size)
    return np.argmax(A, axis = 1)

Using TensorFlow backend.


In [ ]:
%%time
m = 10000
alpha = 0.1
lambd = 0
minibatch_size = 64
iters = 500

X = images.values[:m, :] / 256.
Y = np.zeros((m, 10), dtype = np.int32)
for i in range(m):
    Y[i, labels.values[i, :]] = 1

W = np.random.rand(X.shape[1], 10)
b = np.random.rand(1, 10)

scores = np.array([])

for epoch in range(iters):
    order = np.random.permutation(m)
    epoch_score = 0
    
    for i in range(0, m, minibatch_size):
        W, b, J = update(X[order[i : i + minibatch_size],:],\
                         Y[order[i : i + minibatch_size],:],\
                         W, b, 10, alpha)
        #epoch_score += J
        scores = np.append(scores, J)
    
print(np.sum(predict(X, W, b, 10) == labels.values[:m, :].reshape((m, ))) / m)
plt.plot(scores)